#UniversalSbobinatore

### Installation of Dependencies and Environment Setup
In the next block, we will install the necessary dependencies and set up the environment to run the subsequent code. This includes installing libraries such as Whisper, MoviePy, Transformers, Selenium, and others, as well as configuring the Chromium browser.

In [ ]:
import time
total_time = 0
start_time = time.time()

!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
!pip install moviepy
!pip install --upgrade transformers torch
!pip install torch
!pip install transformers
!pip install selenium webdriver_manager requests
!apt-get update -y
!apt-get install -y chromium-browser
!apt-get install -y chromedriver
!apt install chromium-chromedriver

end_time = time.time()
total_time = total_time + end_time - start_time

# Extracting Audio from Video and Transcription with Whisper

This script uses `moviepy` to extract audio from a video file and save it as an MP3 file, then uses OpenAI's Whisper model for potential transcription. The process includes:

1. Loading the video file in MP4 format.
2. Extracting the audio and saving it as an MP3 file.
3. Optionally, using Whisper for audio transcription (not included in this snippet but could be added).
   
The result is an MP3 file that contains the audio from the video, ready for further processing or transcription.


In [ ]:
# Path to the MP4 file
input_file = "" # NAME OF YOUR VIDEO
from moviepy.editor import *
import whisper
import time

start_time = time.time()

# Load the Whisper model
model = whisper.load_model("small")  # Load the model on the appropriate device

# Name of the output MP3 file
output_file = "audio.mp3"

# Load the video
video = VideoFileClip(input_file)

# Extract the audio and save it as MP3
video.audio.write_audiofile(output_file)

print(f"Conversion completed! File saved as: {output_file}")

# End the timer and calculate the total time
end_time = time.time()
total_time = end_time - start_time
print(f"Total operation time: {total_time:.4f} seconds")


# Audio Transcription with Whisper

This script uses OpenAI's Whisper model to transcribe audio from an MP3 file into text. The process includes:

1. Loading the audio file in MP3 format.
2. Transcribing the audio using the Whisper model.
3. Saving the transcription as a text file.

The result is a text file containing the full transcription of the audio, ready for review or further processing.


In [ ]:
start_time = time.time()
# Path to the MP3 file
input_audio = "audio.mp3"
# Name of the output file
output_text = "trascription.txt"

# Transcribe the audio
result = model.transcribe(input_audio)

# Save the transcription to a text file
with open(output_text, "w", encoding="utf-8") as file:
    file.write(result["text"])

end_time = time.time()
total_time = total_time + end_time - start_time

# Text Summarization with BART

This script uses the BART model from Hugging Face's Transformers library to generate a summary of a given transcription. The process includes:

1. Reading the transcription text from a file.
2. Loading the BART model for summarization (`facebook/bart-large-cnn`).
3. Preparing the content for summarization.

This script sets up the model and tokenizer, ready to summarize the transcription text into a concise version.


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
import torch
import time

# Verifica se è disponibile una GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

start_time = time.time()

# Carica il modello BART e il tokenizer
print("Loading the BART model for summarization...")
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn').to(device)

end_time = time.time()

# Calcola il tempo totale
total_time = end_time - start_time
print(f"Time taken to load the model and process the file: {total_time:.4f} seconds")


# Chunk-Based Summarization of Transcription with BART

This script generates a summary for a large transcription by breaking it into smaller chunks, each of which is summarized separately. The process includes:

1. Splitting the transcription text into chunks of a maximum length (1024 characters).
2. Using the BART model to summarize each chunk with the context of previous summaries.
3. Concatenating the individual summaries to create the final summarized text.
4. Saving the final summary as a text file.

This approach ensures that even long transcriptions can be summarized efficiently while maintaining coherence across chunks.


In [ ]:
# Nome del file da leggere
file_path = "trascription.txt"
# Apri e leggi il contenuto del file
with open(file_path, "r", encoding="utf-8") as file:
    content = file.read()

transcription_text = content
start_time = time.time()  # Start the timer to measure the total operation time.

# Maximum length for each chunk in tokens
max_length = 1024
max_length_chunk = 900  # Maximum length for a single chunk in tokens.

# Tokenize the full transcription text without truncation to calculate its total length.
prompt_tokenized_test = tokenizer(transcription_text, return_tensors="pt", truncation=False, padding=True)
len_p = prompt_tokenized_test["input_ids"].shape[1]  # Get the total number of tokens in the text.

# Calculate the number of chunks based on the maximum chunk length.
N = len_p // max_length_chunk
text_length = len(transcription_text)
chunk_size = text_length // N  # Calculate approximate character length per chunk.

# Split the text into N equal parts.
chunks = [transcription_text[i:i + chunk_size] for i in range(0, text_length, chunk_size)]

# Ensure the last chunk contains any remaining characters.
if len(chunks) > N:
    chunks[-2] += chunks[-1]  # Merge the last chunk with the second last.
    chunks = chunks[:-1]  # Remove the last (now empty) chunk.

# List to store the summaries of each chunk.
summaries = []

# Process each chunk to generate summaries.
for i, chunk in enumerate(chunks):
    # Create a prompt for summarization including the context of the previous summary.
    prompt = f"""
    {chunk}
    """
    # Tokenize the prompt and move it to the GPU.
    prompt_tokenized = tokenizer(prompt, return_tensors="pt", truncation=True, padding="max_length", max_length=1024).to(device)

    minThreshold = int((i / len(chunks)) * 1000)  # Gradually increase the minimum threshold for summary length.

    # Generate the summary using the model with adjusted parameters
    outputs = model.generate(
        prompt_tokenized["input_ids"],
        max_length=1024,  # Maximum length for the summary
        min_length=300,  # Increase the minimum length to ensure more detail
        do_sample=True,  # Enable sampling for more variability in the output
        temperature=0.2,  # Set temperature higher for more diversity in the responses
        top_p=0.9,  # Increase top_p to consider a wider pool of tokens
        length_penalty=1.0,  # Control length penalties to avoid very short outputs
    )

    # Decode the generated summary and append it to the summaries list.
    new_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    summaries.append(new_summary)

    # Update the context with the latest summary for the next chunk.

    print(f"Processed chunk [{i + 1}/{len(chunks)}]")

# End the timer.
end_time = time.time()
# Calculate the total operation time.
total_time = end_time - start_time

# Save all chunk summaries to a separate file.
joined_output_path = "final_summary.txt"
with open(joined_output_path, "w", encoding="utf-8") as file:
    for i, summary in enumerate(summaries, start=1):
        file.write(summary)
        file.write("\n")  # Add a blank line between summaries.
    file.write(f"The total operation is completed in: {int(total_time // 60)} minutes and {total_time % 60:.2f} seconds")


print(f"Summary completed! Final summaries saved in: {joined_output_path}")
